In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt

In [9]:
os.chdir("data/aws_predictions/")

# A549

In [33]:
A549_rep5_run1 = pd.read_csv("A549_rep5_run1_prediction.csv")
A549_rep6_run1 = pd.read_csv("A549_rep6_run1_prediction.csv")

In [69]:
merge1 = A549_rep5_run1.merge(A549_rep6_run1, on = ['transcript','position'], how = 'inner')
merge1['mean_scores_A549'] = merge1[['score_x', 'score_y']].mean(axis=1)
merge1["pred_mean_A549"] = merge1.mean_scores_A549.map(lambda x:1 if x>=0.35 else 0)
merge1 = merge1.drop(columns = ['score_x','score_y'],axis=1)
merge1 = merge1[merge1["pred_mean_A549"] == 1]
merge1.head()

,transcript,position,mean_scores_A549,pred_mean_A549
30,ENST00000371582,875,0.570455,1
58,ENST00000002165,589,0.740822,1
139,ENST00000226299,1874,0.381830,1
190,ENST00000002596,1734,0.527273,1
191,ENST00000002596,1779,0.350012,1


# HCT116

In [35]:
Hct116_rep3_run1 = pd.read_csv("Hct116_rep3_run1_prediction.csv")
Hct116_rep3_run4 = pd.read_csv("Hct116_rep3_run4_prediction.csv")
Hct116_rep4_run3 = pd.read_csv("hct116_rep4_run3_prediction.csv")

In [73]:
mergeHct1 = Hct116_rep3_run1.merge(Hct116_rep3_run4, on = ['transcript','position'],how='inner')
mergeHct2 = mergeHct1.merge(Hct116_rep4_run3, on = ['transcript','position'],how='inner')
mergeHct2 = mergeHct2.rename({'score':'score_z'},axis=1)
mergeHct2['mean_scores_Hct'] = mergeHct2[['score_x', 'score_y','score_z']].mean(axis=1)
mergeHct2["pred_mean_Hct"] = mergeHct2.mean_scores_Hct.map(lambda x:1 if x>=0.35 else 0)
mergeHct2 = mergeHct2.drop(columns = ['score_x','score_y','score_z'],axis=1)
mergeHct2 = mergeHct2[mergeHct2["pred_mean_Hct"] == 1]
mergeHct2.head()

,transcript,position,mean_scores_Hct,pred_mean_Hct
30,ENST00000371582,875,0.448485,1
41,ENST00000002165,1445,0.387879,1
58,ENST00000002165,589,0.535606,1
140,ENST00000226299,1882,0.359091,1
184,ENST00000002596,1280,0.363642,1


# HEPG2

In [37]:
hepG2_rep5_run2 = pd.read_csv("hepG2_rep5_run2_prediction.csv")
hepG2_rep6_run1 = pd.read_csv("hepG2_rep6_run1_prediction.csv")

In [74]:
mergeG2 = hepG2_rep5_run2.merge(hepG2_rep6_run1, on = ['transcript','position'],how='inner')
mergeG2['mean_scores_Hep'] = mergeG2[['score_x', 'score_y']].mean(axis=1)
mergeG2["pred_mean_Hep"] = mergeG2.mean_scores_Hep.map(lambda x:1 if x>=0.35 else 0)
mergeG2 = mergeG2.drop(columns = ['score_x','score_y'],axis=1)
mergeG2 = mergeG2[mergeG2["pred_mean_Hep"] == 1]
mergeG2.head()

,transcript,position,mean_scores_Hep,pred_mean_Hep
9,ENST00000373020,936,0.381818,1
30,ENST00000371582,875,0.515909,1
41,ENST00000002165,1445,0.368182,1
58,ENST00000002165,589,0.543182,1
139,ENST00000226299,1874,0.365909,1


# K562

In [40]:
k562_rep4_run1 = pd.read_csv("k562_rep4_run1_prediction.csv")
k562_rep5_run1 = pd.read_csv("k562_rep5_run1_prediction.csv")
k562_rep6_run1 = pd.read_csv("k562_rep6_run1_prediction.csv")

In [75]:
mergek562_1 = k562_rep4_run1.merge(k562_rep5_run1, on = ['transcript','position'],how='inner')
mergek562_2 = mergek562_1.merge(k562_rep6_run1, on = ['transcript','position'],how='inner')
mergek562_2 = mergek562_2.rename({'score':'score_z'},axis=1)
mergek562_2['mean_scores_K562'] = mergek562_2[['score_x', 'score_y','score_z']].mean(axis=1)
mergek562_2["pred_mean_K562"] = mergek562_2.mean_scores_K562.map(lambda x:1 if x>=0.35 else 0)
mergek562_2 = mergek562_2.drop(columns = ['score_x','score_y','score_z'],axis=1)
mergek562_2 = mergek562_2[mergek562_2["pred_mean_K562"] == 1]
mergek562_2.head()

,transcript,position,mean_scores_K562,pred_mean_K562
17,ENST00000371582,875,0.353030,1
28,ENST00000002165,1445,0.398485,1
45,ENST00000002165,589,0.637762,1
227,ENST00000000412,2440,0.565152,1
232,ENST00000000412,2580,0.518182,1


# MCF7

In [42]:
mcf7_rep3_run1 = pd.read_csv("mcf7_rep3_run1_prediction.csv")
mcf7_rep4_run1 = pd.read_csv("mcf7_rep4_run1_prediction.csv")

In [76]:
mergemcf7 = mcf7_rep3_run1.merge(mcf7_rep4_run1, on = ['transcript','position'],how='inner')
mergemcf7['mean_scores_MCF7'] = mergemcf7[['score_x', 'score_y']].mean(axis=1)
mergemcf7["pred_mean_MCF7"] = mergemcf7.mean_scores_MCF7.map(lambda x:1 if x>=0.35 else 0)
mergemcf7 = mergemcf7.drop(columns = ['score_x','score_y'],axis=1)
mergemcf7 = mergemcf7[mergemcf7["pred_mean_MCF7"] == 1]
mergemcf7.head()

,transcript,position,mean_scores_MCF7,pred_mean_MCF7
30,ENST00000371582,875,0.522727,1
41,ENST00000002165,1445,0.404545,1
58,ENST00000002165,589,0.559091,1
258,ENST00000000412,2440,0.559091,1
263,ENST00000000412,2580,0.468182,1


In [80]:
all_cells = [merge1,mergeG2,mergeHct2,mergek562_2,mergemcf7]
all_cells_df = pd.concat(all_cells,axis=1)

In [85]:
##compare how many number of cancer cell with same transcript and position in A549 and Hep
A549_with_Hep_cell = [merge1,mergeG2]
A549_with_Hep_cell_df = pd.concat(A549_with_Hep_cell,axis=1)
A549_with_Hep_cell_df = A549_with_Hep_cell_df.dropna()
A549_with_Hep_cell_df.count()


transcript          139
position            139
mean_scores_A549    139
pred_mean_A549      139
transcript          139
position            139
mean_scores_Hep     139
pred_mean_Hep       139
dtype: int64

In [86]:
A549_with_Hct_cell = [merge1,mergeHct2]
A549_with_Hct_cell_df = pd.concat(A549_with_Hct_cell,axis=1)
A549_with_Hct_cell_df = A549_with_Hct_cell_df.dropna()
A549_with_Hct_cell_df.count()

transcript          183
position            183
mean_scores_A549    183
pred_mean_A549      183
transcript          183
position            183
mean_scores_Hct     183
pred_mean_Hct       183
dtype: int64

In [87]:
A549_with_K562_cell = [merge1,mergek562_2]
A549_with_K562_cell_df = pd.concat(A549_with_K562_cell,axis=1)
A549_with_K562_cell_df = A549_with_K562_cell_df.dropna()
A549_with_K562_cell_df.count()

transcript          87
position            87
mean_scores_A549    87
pred_mean_A549      87
transcript          87
position            87
mean_scores_K562    87
pred_mean_K562      87
dtype: int64

In [88]:
A549_with_MCF7_cell = [merge1,mergemcf7]
A549_with_MCF7_cell_df = pd.concat(A549_with_MCF7_cell,axis=1)
A549_with_MCF7_cell_df = A549_with_MCF7_cell_df.dropna()
A549_with_MCF7_cell_df.count()

transcript          166
position            166
mean_scores_A549    166
pred_mean_A549      166
transcript          166
position            166
mean_scores_MCF7    166
pred_mean_MCF7      166
dtype: int64

In [89]:
##time to compare with Hct
A549_with_Hct_cell = [merge1,mergeHct2]
A549_with_Hct_cell_df = pd.concat(A549_with_Hct_cell,axis=1)
A549_with_Hct_cell_df = A549_with_Hct_cell_df.dropna()
A549_with_Hct_cell_df.count()

transcript          183
position            183
mean_scores_A549    183
pred_mean_A549      183
transcript          183
position            183
mean_scores_Hct     183
pred_mean_Hct       183
dtype: int64

In [90]:
Hct_with_Hep = [mergeHct2,mergeG2]
Hct_with_Hep_df = pd.concat(Hct_with_Hep,axis=1)
Hct_with_Hep_df = Hct_with_Hep_df.dropna()
Hct_with_Hep_df.count()

transcript         156
position           156
mean_scores_Hct    156
pred_mean_Hct      156
transcript         156
position           156
mean_scores_Hep    156
pred_mean_Hep      156
dtype: int64

In [91]:
Hct_with_K562 = [mergeHct2,mergek562_2]
Hct_with_K562_df = pd.concat(Hct_with_K562,axis=1)
Hct_with_K562_df = Hct_with_K562_df.dropna()
Hct_with_K562_df.count()

transcript          109
position            109
mean_scores_Hct     109
pred_mean_Hct       109
transcript          109
position            109
mean_scores_K562    109
pred_mean_K562      109
dtype: int64

In [92]:
Hct_with_MCF7 = [mergeHct2,mergemcf7]
Hct_with_MCF7_df = pd.concat(Hct_with_MCF7,axis=1)
Hct_with_MCF7_df = Hct_with_MCF7_df.dropna()
Hct_with_MCF7_df.count()

transcript          199
position            199
mean_scores_Hct     199
pred_mean_Hct       199
transcript          199
position            199
mean_scores_MCF7    199
pred_mean_MCF7      199
dtype: int64

In [93]:
##time to compare for HepG2
A549_with_Hep_cell = [merge1,mergeG2]
A549_with_Hep_cell_df = pd.concat(A549_with_Hep_cell,axis=1)
A549_with_Hep_cell_df = A549_with_Hep_cell_df.dropna()
A549_with_Hep_cell_df.count()

transcript          139
position            139
mean_scores_A549    139
pred_mean_A549      139
transcript          139
position            139
mean_scores_Hep     139
pred_mean_Hep       139
dtype: int64

In [94]:
Hct_with_Hep = [mergeHct2,mergeG2]
Hct_with_Hep_df = pd.concat(Hct_with_Hep,axis=1)
Hct_with_Hep_df = Hct_with_Hep_df.dropna()
Hct_with_Hep_df.count()

transcript         156
position           156
mean_scores_Hct    156
pred_mean_Hct      156
transcript         156
position           156
mean_scores_Hep    156
pred_mean_Hep      156
dtype: int64

In [95]:
Hep_with_K562 = [mergeG2,mergek562_2]
Hep_with_K562_df = pd.concat(Hep_with_K562,axis=1)
Hep_with_K562_df = Hep_with_K562_df.dropna()
Hep_with_K562_df.count()

transcript          103
position            103
mean_scores_Hep     103
pred_mean_Hep       103
transcript          103
position            103
mean_scores_K562    103
pred_mean_K562      103
dtype: int64

In [97]:
Hep_with_MCF7 = [mergeG2,mergemcf7]
Hep_with_MCF7_df = pd.concat(Hep_with_MCF7,axis=1)
Hep_with_MCF7_df = Hep_with_MCF7_df.dropna()
Hep_with_MCF7_df.count()

transcript          167
position            167
mean_scores_Hep     167
pred_mean_Hep       167
transcript          167
position            167
mean_scores_MCF7    167
pred_mean_MCF7      167
dtype: int64

In [98]:
##K562 comparison
A549_with_K562_cell = [merge1,mergek562_2]
A549_with_K562_cell_df = pd.concat(A549_with_K562_cell,axis=1)
A549_with_K562_cell_df = A549_with_K562_cell_df.dropna()
A549_with_K562_cell_df.count()

transcript          87
position            87
mean_scores_A549    87
pred_mean_A549      87
transcript          87
position            87
mean_scores_K562    87
pred_mean_K562      87
dtype: int64

In [99]:
Hct_with_K562 = [mergeHct2,mergek562_2]
Hct_with_K562_df = pd.concat(Hct_with_K562,axis=1)
Hct_with_K562_df = Hct_with_K562_df.dropna()
Hct_with_K562_df.count()

transcript          109
position            109
mean_scores_Hct     109
pred_mean_Hct       109
transcript          109
position            109
mean_scores_K562    109
pred_mean_K562      109
dtype: int64

In [100]:
Hep_with_K562 = [mergeG2,mergek562_2]
Hep_with_K562_df = pd.concat(Hep_with_K562,axis=1)
Hep_with_K562_df = Hep_with_K562_df.dropna()
Hep_with_K562_df.count()

transcript          103
position            103
mean_scores_Hep     103
pred_mean_Hep       103
transcript          103
position            103
mean_scores_K562    103
pred_mean_K562      103
dtype: int64

In [101]:
K562_with_MCF7 = [mergek562_2,mergemcf7]
K562_with_MCF7_df = pd.concat(K562_with_MCF7,axis=1)
K562_with_MCF7_df = K562_with_MCF7_df.dropna()
K562_with_MCF7_df.count()

transcript          117
position            117
mean_scores_K562    117
pred_mean_K562      117
transcript          117
position            117
mean_scores_MCF7    117
pred_mean_MCF7      117
dtype: int64

In [102]:
##MCF7 comparison
A549_with_MCF7_cell = [merge1,mergemcf7]
A549_with_MCF7_cell_df = pd.concat(A549_with_MCF7_cell,axis=1)
A549_with_MCF7_cell_df = A549_with_MCF7_cell_df.dropna()
A549_with_MCF7_cell_df.count()

transcript          166
position            166
mean_scores_A549    166
pred_mean_A549      166
transcript          166
position            166
mean_scores_MCF7    166
pred_mean_MCF7      166
dtype: int64

In [103]:
Hct_with_MCF7 = [mergeHct2,mergemcf7]
Hct_with_MCF7_df = pd.concat(Hct_with_MCF7,axis=1)
Hct_with_MCF7_df = Hct_with_MCF7_df.dropna()
Hct_with_MCF7_df.count()

transcript          199
position            199
mean_scores_Hct     199
pred_mean_Hct       199
transcript          199
position            199
mean_scores_MCF7    199
pred_mean_MCF7      199
dtype: int64

In [104]:
Hep_with_MCF7 = [mergeG2,mergemcf7]
Hep_with_MCF7_df = pd.concat(Hep_with_MCF7,axis=1)
Hep_with_MCF7_df = Hep_with_MCF7_df.dropna()
Hep_with_MCF7_df.count()

transcript          167
position            167
mean_scores_Hep     167
pred_mean_Hep       167
transcript          167
position            167
mean_scores_MCF7    167
pred_mean_MCF7      167
dtype: int64

In [105]:
K562_with_MCF7 = [mergek562_2,mergemcf7]
K562_with_MCF7_df = pd.concat(K562_with_MCF7,axis=1)
K562_with_MCF7_df = K562_with_MCF7_df.dropna()
K562_with_MCF7_df.count()

transcript          117
position            117
mean_scores_K562    117
pred_mean_K562      117
transcript          117
position            117
mean_scores_MCF7    117
pred_mean_MCF7      117
dtype: int64